# PyWry Toolbar System Demo

This notebook demonstrates PyWry's powerful toolbar system. **Most examples require zero custom HTML or JavaScript** - everything is done with declarative components and built-in events.

## Built-in Events (No Custom JS Needed!)

| Event | Description |
|-------|-------------|
| `pywry:set-content` | Update element innerHTML/textContent by id or selector |
| `pywry:set-style` | Update element styles by id or selector |
| `pywry:inject-css` | Dynamically inject CSS |
| `pywry:update-theme` | Switch between dark/light mode |

## What You'll Learn

1. **Basic Toolbar** - Counter with button click handler
2. **Input Components** - Text, number, select, slider with 2-way binding
3. **Multiple Positions** - All 7 toolbar positions (header, footer, top, bottom, left, right, inside)
4. **Div Components** - Labels, separators, and layout helpers
5. **Collapsible & Resizable** - Interactive sidebar with collapse/resize features
6. **Product Dashboard** - Dynamic list/grid rendering with filtering and animations
7. **Marquee Ticker** - Scrolling content with dynamic updates
8. **All Components** - Complete component showcase with every input type

In [ ]:
from pywry import (
    Button,
    Checkbox,
    DateInput,
    Div,
    MultiSelect,
    NumberInput,
    Option,
    PyWry,
    RadioGroup,
    RangeInput,
    SecretInput,
    Select,
    SliderInput,
    TabGroup,
    TextArea,
    TextInput,
    Toggle,
    Toolbar,
)

app = PyWry()

---

## 1. Basic Toolbar - Button Click Handler

The simplest toolbar - a counter using only **toolbar components** and built-in `pywry:set-content` event.
No custom HTML or JavaScript needed!

In [ ]:
# State for tracking clicks
click_count = 0

# Create a simple toolbar with one button
basic_toolbar = Toolbar(
    position="top",
    items=[
        Button(label="Click Me!", event="app:click", variant="neutral"),
        Button(label="Reset", event="app:reset", variant="primary"),
    ],
)


def on_button_click(data, event_type, label):
    """Handle button click - update the displayed count."""
    global click_count
    click_count += 1
    # Use built-in pywry:set-content - no custom JS needed!
    basic_widget.emit("pywry:set-content", {"id": "counter", "text": str(click_count)})


def on_reset(data, event_type, label):
    """Reset the counter."""
    global click_count
    click_count = 0
    basic_widget.emit("pywry:set-content", {"id": "counter", "text": "0"})


basic_widget = app.show(
    Div(
        content="0",
        component_id="counter",
        style="width: 100%; text-align: center; justify-content: center; display: flex; align-items: center; font-size: 48px; font-weight: bold;",
    ).build_html(),
    title="Basic Toolbar Demo",
    toolbars=[basic_toolbar],
    callbacks={
        "app:click": on_button_click,
        "app:reset": on_reset,
    },
    height=100,
)

---

## 2. Input Components with 2-Way Binding

Various input types that emit events when values change. Uses built-in `pywry:set-content` and `pywry:set-style` events - no custom JavaScript!

In [ ]:
# Color map for CSS
COLOR_MAP = {
    "blue": "#0078d4",
    "red": "#dc3545",
    "green": "#28a745",
    "purple": "#6f42c1",
}


def on_name_change(data, event_type, label):
    """Handle text input change - update greeting text."""
    name = data.get("value", "World")
    inputs_widget.emit(
        "pywry:set-content", {"id": "greeting", "text": f"Hello, {name}!"}
    )


def on_size_change(data, event_type, label):
    """Handle number input change - update font size."""
    size = data.get("value", 24)
    inputs_widget.emit(
        "pywry:set-style", {"id": "greeting", "styles": {"fontSize": f"{size}px"}}
    )


def on_color_change(data, event_type, label):
    """Handle select change - update text color."""
    color = COLOR_MAP.get(data.get("value", "blue"), "#0078d4")
    inputs_widget.emit(
        "pywry:set-style", {"id": "greeting", "styles": {"color": color}}
    )


def on_opacity_change(data, event_type, label):
    """Handle slider change - update opacity."""
    opacity = data.get("value", 100) / 100
    inputs_widget.emit(
        "pywry:set-style", {"id": "greeting", "styles": {"opacity": str(opacity)}}
    )


# Create toolbar with various input types
inputs_toolbar = Toolbar(
    position="top",
    items=[
        TextInput(
            label="Name:",
            event="form:name",
            value="World",
            placeholder="Enter a name...",
            debounce=300,
        ),
        NumberInput(label="Size:", event="form:size", value=24, min=8, max=72, step=2),
        Select(
            label="Color:",
            event="form:color",
            options=[
                Option(label="Blue", value="blue"),
                Option(label="Red", value="red"),
                Option(label="Green", value="green"),
                Option(label="Purple", value="purple"),
            ],
            selected="blue",
        ),
        SliderInput(
            label="Opacity:",
            event="form:opacity",
            value=100,
            min=0,
            max=100,
            step=5,
            show_value=True,
        ),
    ],
)

content_html = Div(
    content="<h1 id='greeting' style='font-size: 24px; color: #0078d4; margin: 0;'>Hello, World!</h1>",
    style="width: 100%; text-align: center; padding: 20px 0; justify-content: center; display: flex; align-items: center;",
).build_html()


inputs_widget = app.show(
    content_html,
    title="Input Components Demo",
    toolbars=[inputs_toolbar],
    callbacks={
        "form:name": on_name_change,
        "form:size": on_size_change,
        "form:color": on_color_change,
        "form:opacity": on_opacity_change,
    },
    height=100,
)

---

## 3. Multiple Toolbars with Different Positions

Toolbars can be positioned at:
- **header** - Full-width bar at very top (outside left/right)
- **footer** - Full-width bar at very bottom (outside left/right)
- **top** - Horizontal bar (inside left/right columns)
- **bottom** - Horizontal bar (inside left/right columns)
- **left** - Vertical bar extending from header to footer
- **right** - Vertical bar extending from header to footer
- **inside** - Floating overlay (absolute positioned)

Layout structure:
```
HEADER (full width)
LEFT | TOP / CONTENT / BOTTOM | RIGHT
FOOTER (full width)
```

In [ ]:
def on_position_click(data, event_type, label):
    """Handle position button clicks - update display with built-in event."""
    position = data.get("position", "unknown")
    positions_widget.emit(
        "pywry:set-content", {"id": "last-click", "text": f"Last click: {position}"}
    )


# Create toolbars for each position
header_toolbar = Toolbar(
    position="header",
    items=[
        Div(
            content="<strong>HEADER</strong> - Full width at very top",
            style="width: 100%; text-align: center;",
        )
    ],
)

top_toolbar = Toolbar(
    position="top",
    items=[
        Button(
            label="TOP", event="pos:click", data={"position": "top"}, variant="neutral"
        ),
        Div(
            content="<span style='color: var(--pywry-text-secondary);'>Inside left/right columns</span>"
        ),
    ],
    style="justify-content: center;",
)

# Left toolbar - resizable, title at top, buttons centered vertically
left_toolbar = Toolbar(
    position="left",
    resizable=True,
    items=[
        Div(content="<strong>LEFT</strong>", style="text-align: center; width: 100%;"),
        Div(content="", style="flex: 1;"),  # Top spacer
        Button(
            label="L1",
            event="pos:click",
            data={"position": "left-1"},
            style="width: 100%;",
        ),
        Button(
            label="L2",
            event="pos:click",
            data={"position": "left-2"},
            variant="danger",
            style="width: 100%;",
        ),
        Div(content="", style="flex: 1;"),  # Bottom spacer
    ],
)

# Right toolbar - resizable, title at top, buttons centered vertically
right_toolbar = Toolbar(
    position="right",
    resizable=True,
    items=[
        Div(content="<strong>RIGHT</strong>", style="text-align: center; width: 100%;"),
        Div(content="", style="flex: 1;"),  # Top spacer
        Button(
            label="R1",
            event="pos:click",
            data={"position": "right-1"},
            style="width: 100%;",
        ),
        Button(
            label="R2",
            event="pos:click",
            data={"position": "right-2"},
            variant="secondary",
            style="width: 100%;",
        ),
        Div(content="", style="flex: 1;"),  # Bottom spacer
    ],
)

bottom_toolbar = Toolbar(
    position="bottom",
    items=[
        Button(
            label="BOTTOM",
            event="pos:click",
            data={"position": "bottom"},
            variant="outline",
        ),
        Div(
            content="<span style='color: var(--pywry-text-secondary);'>Inside left/right columns</span>"
        ),
    ],
    style="justify-content: center;",
    resizable=True,
)

# Inside toolbar 1 - default position (top-right)
inside_toolbar = Toolbar(
    position="inside",
    items=[
        Button(
            label="⚙️ INSIDE (top-right)",
            event="pos:click",
            data={"position": "inside-1"},
            variant="ghost",
            size="sm",
        )
    ],
)

# Inside toolbar 2 - custom position (bottom-left)
inside_toolbar_2 = Toolbar(
    position="inside",
    items=[
        Button(
            label="📍 INSIDE (bottom-left)",
            event="pos:click",
            data={"position": "inside-2"},
            variant="outline",
            size="sm",
        )
    ],
    style="top: auto; right: auto; bottom: 10px; left: 10px;",
)

footer_toolbar = Toolbar(
    position="footer",
    items=[
        Div(
            content="<strong>FOOTER</strong>: Click any button...",
            style="width: 100%; text-align: center; color: var(--pywry-text-secondary); margin-right: 25px; justify-content: right;",
            component_id="last-click",
        )
    ],
)

# Content area - use Div to create centered content
content_html = Div(
    content="CONTENT AREA<br>(scrollable)<br><br>Drag edges to resize LEFT/RIGHT",
    style="display: flex; justify-content: center; align-items: center; height: 100%; text-align: center; color: var(--pywry-text-secondary)",
).build_html()

positions_widget = app.show(
    content_html,
    title="All 7 Toolbar Positions",
    toolbars=[
        header_toolbar,  # Very top, full width
        footer_toolbar,  # Very bottom, full width
        left_toolbar,  # Left side, resizable
        right_toolbar,  # Right side, resizable
        top_toolbar,  # Top, inside left/right
        bottom_toolbar,  # Bottom, inside left/right
        inside_toolbar,  # Floating overlay (top-right default)
        inside_toolbar_2,  # Floating overlay (bottom-left custom)
    ],
    callbacks={"pos:click": on_position_click},
    height=400,
)

---

## 4. Div Components - Labels, Separators & Layout

The `Div` component adds non-interactive elements to toolbars: labels, separators, status displays.
Combined with built-in events, creates app-like layouts with zero custom JavaScript.

In [ ]:
# Track current file state
current_file = {"name": "Untitled", "saved": True}


def on_file_action(data, event_type, label):
    """Handle file actions."""
    action = data.get("action", "")
    if action == "new":
        current_file["name"] = "Untitled"
        current_file["saved"] = False
    elif action == "save":
        current_file["saved"] = True

    # Update display using built-in events
    div_widget.emit(
        "pywry:set-content", {"id": "file-name", "text": current_file["name"]}
    )
    status = "● Saved" if current_file["saved"] else "○ Unsaved"
    color = "var(--pywry-text-secondary)" if current_file["saved"] else "#ef4444"
    div_widget.emit("pywry:set-content", {"id": "file-status", "text": status})
    div_widget.emit(
        "pywry:set-style", {"id": "file-status", "styles": {"color": color}}
    )


def on_view_change(data, event_type, label):
    """Handle view toggle."""
    view = data.get("view", "edit").capitalize()
    div_widget.emit("pywry:set-content", {"id": "view-mode", "text": f"Mode: {view}"})


# Toolbar using Div to create logical groups
div_toolbar = Toolbar(
    position="top",
    items=[
        Div(content="<strong>File:</strong>", style="margin-right: 4px;"),
        Button(
            label="New", event="app:file", data={"action": "new"}, variant="secondary"
        ),
        Button(
            label="Save", event="app:file", data={"action": "save"}, variant="neutral"
        ),
        # Separator
        Div(
            content="",
            style="width: 1px; height: 24px; background: var(--pywry-border-color); margin: 0 12px;",
        ),
        Div(content="<strong>View:</strong>", style="margin-right: 4px;"),
        Button(
            label="Edit", event="app:view", data={"view": "edit"}, variant="outline"
        ),
        Button(
            label="Preview",
            event="app:view",
            data={"view": "preview"},
            variant="outline",
        ),
    ],
)

div_widget = app.show(
    Div(
        content="""
            <span id='file-name' style='font-weight: bold;'>Untitled</span>
            <span id='file-status' style='margin-left: 8px; color: var(--pywry-text-secondary);'>● Saved</span>
            <span id='view-mode' style='margin-left: 16px; color: var(--pywry-accent);'>Mode: Edit</span>
        """,
        style="width: 100%; text-align: center; justify-content: center; height: 100%;",
    ).build_html(),
    title="Div for Toolbar Layout",
    toolbars=[div_toolbar],
    callbacks={
        "app:file": on_file_action,
        "app:view": on_view_change,
    },
    height=100,
)

---

## 5. Collapsible & Resizable Toolbars

Toolbars can be made collapsible and resizable. State persists in `sessionStorage`.
This example uses only built-in events - no custom JavaScript!

In [ ]:
# Page icons and names
PAGES = {
    "dashboard": ("📊", "Dashboard"),
    "files": ("📁", "Files"),
    "settings": ("⚙️", "Settings"),
}

# Theme background colors
THEME_COLORS = {
    "Default": "var(--pywry-bg-secondary)",
    "Blue": "rgba(0, 120, 212, 0.15)",
    "Green": "rgba(40, 167, 69, 0.15)",
    "Purple": "rgba(111, 66, 193, 0.15)",
}


def on_toolbar_collapse(data, event_type, label):
    """Handle toolbar collapse/expand events."""
    state = "collapsed" if data.get("collapsed") else "expanded"
    interactive_widget.emit(
        "pywry:set-content", {"id": "status", "text": f"Sidebar {state}"}
    )


def on_toolbar_resize(data, event_type, label):
    """Handle toolbar resize events."""
    width = data.get("width", 0)
    interactive_widget.emit(
        "pywry:set-content", {"id": "status", "text": f"Sidebar width: {width}px"}
    )


def on_sidebar_item_click(data, event_type, label):
    """Handle sidebar item click."""
    item = data.get("item", "dashboard")
    icon, name = PAGES.get(item, ("📊", "Dashboard"))
    interactive_widget.emit(
        "pywry:set-content", {"id": "current-page", "text": f"{icon} {name}"}
    )
    interactive_widget.emit(
        "pywry:set-content", {"id": "status", "text": f"Viewing {name}"}
    )


def on_action_click(data, event_type, label):
    """Handle action button clicks."""
    action = data.get("action", "")
    if action == "alert":
        interactive_widget.emit(
            "pywry:set-content", {"id": "status", "text": "🔔 Alert triggered!"}
        )
    elif action == "color":
        import random

        colors = ["#0078d4", "#28a745", "#6f42c1", "#dc3545", "#fd7e14"]
        interactive_widget.emit(
            "pywry:set-style",
            {"id": "current-page", "styles": {"color": random.choice(colors)}},
        )
        interactive_widget.emit(
            "pywry:set-content", {"id": "status", "text": "Color changed!"}
        )


def on_theme_change(data, event_type, label):
    """Handle theme select change."""
    theme = data.get("value", "Default")
    bg = THEME_COLORS.get(theme, THEME_COLORS["Default"])
    interactive_widget.emit(
        "pywry:set-style", {"id": "content-area", "styles": {"background": bg}}
    )
    interactive_widget.emit(
        "pywry:set-content", {"id": "status", "text": f"Theme: {theme}"}
    )


# Header toolbar
header = Toolbar(
    position="header",
    items=[
        Div(
            content="<span id='current-page' style='font-weight: 600;'>📊 Dashboard</span>",
            style="flex: 1;",
        ),
        Div(
            content="<span id='status' style='color: var(--pywry-text-secondary);'>Select from sidebar</span>"
        ),
    ],
)

# Bottom toolbar with actions
bottom_toolbar = Toolbar(
    position="bottom",
    items=[
        Button(
            label="Alert",
            event="panel:action",
            data={"action": "alert"},
            variant="neutral",
        ),
        Button(
            label="Color",
            event="panel:action",
            data={"action": "color"},
            variant="secondary",
        ),
        Select(
            label="Theme:",
            event="panel:theme",
            options=["Default", "Blue", "Green", "Purple"],
            selected="Default",
        ),
    ],
)

# Resizable left sidebar
sidebar = Toolbar(
    position="left",
    resizable=True,
    collapsible=True,
    items=[
        Div(
            content="<strong>Navigation</strong>",
            style="border-bottom: 1px solid var(--pywry-border-color); padding-bottom: 4px; margin-bottom: 4px;",
        ),
        Button(
            label="📊 Dashboard",
            event="sidebar:item",
            data={"item": "dashboard"},
            variant="ghost",
        ),
        Button(
            label="📁 Files",
            event="sidebar:item",
            data={"item": "files"},
            variant="ghost",
        ),
        Button(
            label="⚙️ Settings",
            event="sidebar:item",
            data={"item": "settings"},
            variant="ghost",
        ),
    ],
)

# Content wrapper with padding to inset the styled content area
content_wrapper = Div(
    content=Div(
        content="• Resize sidebar by dragging edge<br>• Collapse with arrow button<br>• Click items to navigate",
        style="display: flex; flex-direction: column; justify-content: center; align-items: center; padding: 16px; background: var(--pywry-bg-secondary); border-radius: 6px; text-align: center; height: 100%; box-sizing: border-box;",
        component_id="content-area",
    ).build_html(),
    style="display: flex; justify-content: center; align-items: center; padding: 12px; height: 100%; box-sizing: border-box;",
).build_html()

interactive_widget = app.show(
    content_wrapper,
    title="Collapsible & Resizable Demo",
    toolbars=[header, sidebar, bottom_toolbar],
    callbacks={
        "toolbar:collapse": on_toolbar_collapse,
        "toolbar:expand": on_toolbar_collapse,
        "toolbar:resize": on_toolbar_resize,
        "sidebar:item": on_sidebar_item_click,
        "panel:action": on_action_click,
        "panel:theme": on_theme_change,
    },
    height=200,
)

---

## 6. Product Dashboard - Dynamic List/Grid Rendering

A complete filterable product grid demonstrating:
- **Dynamic HTML generation** with `Div` components and `build_html()`
- **Real-time filtering** via search, category, and price range
- **CSS hover animations** using `HtmlContent` with `inline_css`
- **Grid updates** using built-in `pywry:set-content` event

All rendering is done in Python - no custom JavaScript!

In [ ]:
# Dashboard state
dashboard_state = {
    "search": "",
    "category": "all",
    "price_min": 0,
    "price_max": 1000,
    "sort": "name",
    "items": [
        {"name": "Laptop Pro", "category": "electronics", "price": 999},
        {"name": "Wireless Mouse", "category": "electronics", "price": 29},
        {"name": "Desk Chair", "category": "furniture", "price": 299},
        {"name": "Standing Desk", "category": "furniture", "price": 499},
        {"name": "Notebook Set", "category": "office", "price": 15},
        {"name": 'Monitor 27"', "category": "electronics", "price": 399},
        {"name": "Bookshelf", "category": "furniture", "price": 149},
        {"name": "Pen Holder", "category": "office", "price": 12},
    ],
}

# Category colors for badges
CATEGORY_COLORS = {
    "electronics": "#0078d4",
    "furniture": "#28a745",
    "office": "#6f42c1",
}


def render_item(item):
    """Render a single item card using Div components."""
    color = CATEGORY_COLORS.get(item["category"], "#666")
    return Div(
        class_name="product-card",
        style="background: var(--pywry-bg-secondary); border-radius: 8px; padding: 12px; border: 1px solid var(--pywry-border-color); display: flex; flex-direction: column; transition: transform 0.15s ease, box-shadow 0.15s ease, border-color 0.15s ease; cursor: pointer;",
        children=[
            Div(
                content=item["name"],
                style="margin: 0 0 8px 0; font-size: 14px; font-weight: 600;",
            ),
            Div(
                children=[
                    Div(
                        content=item["category"],
                        style=f"display: inline-block; background: {color}; color: white; padding: 2px 6px; border-radius: 4px; font-size: 11px;",
                    ),
                ],
            ),
            Div(
                content=f"${item['price']}",
                style="margin: 8px 0 0 0; font-size: 18px; font-weight: bold; color: var(--pywry-accent);",
            ),
        ],
    ).build_html()


# CSS for hover effects
CARD_HOVER_CSS = """
.product-card:hover {
    transform: translateY(-2px);
    box-shadow: 0 4px 12px rgba(0, 0, 0, 0.15);
    border-color: var(--pywry-accent);
}
"""


def render_grid(items):
    """Render the entire grid as HTML."""
    if not items:
        return Div(
            content="No items match your filters",
            style="grid-column: 1/-1; text-align: center; padding: 40px; color: var(--pywry-text-secondary);",
        ).build_html()
    return "".join(render_item(item) for item in items)


def filter_items():
    """Filter items based on current state."""
    items = dashboard_state["items"]
    search = dashboard_state["search"].lower()
    category = dashboard_state["category"]
    price_min = dashboard_state["price_min"]
    price_max = dashboard_state["price_max"]

    filtered = [
        item
        for item in items
        if (search == "" or search in item["name"].lower())
        and (category == "all" or item["category"] == category)
        and (price_min <= item["price"] <= price_max)
    ]

    # Sort
    sort_key = dashboard_state["sort"]
    filtered.sort(key=lambda x: x.get(sort_key, ""))

    return filtered


def update_dashboard():
    """Update the dashboard using built-in pywry:set-content - no custom JS needed!"""
    filtered = filter_items()
    total = len(dashboard_state["items"])

    # Update grid content
    dashboard_widget.emit(
        "pywry:set-content", {"id": "items-container", "html": render_grid(filtered)}
    )
    # Update count
    dashboard_widget.emit(
        "pywry:set-content",
        {"id": "result-count", "text": f"Showing {len(filtered)} of {total} items"},
    )


# Handlers
def on_search(data, event_type, label):
    dashboard_state["search"] = data.get("value", "")
    update_dashboard()


def on_category(data, event_type, label):
    dashboard_state["category"] = data.get("value", "all")
    update_dashboard()


def on_price_range(data, event_type, label):
    dashboard_state["price_min"] = data.get("start", 0)
    dashboard_state["price_max"] = data.get("end", 1000)
    update_dashboard()


def on_sort(data, event_type, label):
    dashboard_state["sort"] = data.get("value", "name")
    update_dashboard()


def on_reset_filters(data, event_type, label):
    """Reset all filters to defaults."""
    dashboard_state["search"] = ""
    dashboard_state["category"] = "all"
    dashboard_state["price_min"] = 0
    dashboard_state["price_max"] = 1000
    dashboard_state["sort"] = "name"
    update_dashboard()


# Header with title
header_toolbar = Toolbar(
    position="header",
    items=[
        Div(
            content="<strong style='font-size: 16px;'>📦 Product Dashboard</strong>",
            style="padding: 4px 8px;",
        ),
    ],
)

# Top toolbar with search and category filter
search_toolbar = Toolbar(
    position="top",
    items=[
        TextInput(
            label="Search:",
            event="dash:search",
            placeholder="Filter by name...",
            debounce=200,
        ),
        Select(
            label="Category:",
            event="dash:category",
            options=[
                Option(label="All", value="all"),
                Option(label="Electronics", value="electronics"),
                Option(label="Furniture", value="furniture"),
                Option(label="Office", value="office"),
            ],
            selected="all",
        ),
        Select(
            label="Sort:",
            event="dash:sort",
            options=[
                Option(label="Name", value="name"),
                Option(label="Price", value="price"),
                Option(label="Category", value="category"),
            ],
            selected="name",
        ),
    ],
)

# Bottom toolbar with RangeInput for price filtering
price_toolbar = Toolbar(
    position="bottom",
    items=[
        RangeInput(
            label="Price Range:",
            event="dash:price",
            min=0,
            max=1000,
            step=10,
            start=0,
            end=1000,
            show_value=True,
        ),
        Button(label="Reset Filters", event="dash:reset", variant="secondary"),
    ],
)

# Footer with count
footer_toolbar = Toolbar(
    position="footer",
    items=[
        Div(
            content="<span id='result-count'>Showing 8 of 8 items</span>",
            style="padding: 4px 8px; color: var(--pywry-text-secondary);",
        ),
    ],
)

# Initial grid - all rendered with Div components!
initial_grid = render_grid(dashboard_state["items"])

# Use HtmlContent to combine HTML + inline CSS
from pywry import HtmlContent

dashboard_widget = app.show(
    HtmlContent(
        html=Div(
            content=initial_grid,
            component_id="items-container",
            style="display: grid; grid-template-columns: repeat(auto-fill, minmax(180px, 1fr)); gap: 12px; padding: 8px;",
        ).build_html(),
        inline_css=CARD_HOVER_CSS,
    ),
    title="Product Dashboard",
    toolbars=[header_toolbar, search_toolbar, price_toolbar, footer_toolbar],
    callbacks={
        "dash:search": on_search,
        "dash:category": on_category,
        "dash:price": on_price_range,
        "dash:sort": on_sort,
        "dash:reset": on_reset_filters,
    },
    height=400,
)

---

## 7. Marquee Component - Scrolling Ticker

The `Marquee` component creates a smooth scrolling text or content ticker:

- **text** - Simple text content to scroll
- **speed** - Animation duration in seconds (lower = faster)
- **direction** - "left", "right", "up", or "down"
- **behavior** - "scroll" (loop), "alternate" (bounce), or "slide" (once)
- **pause_on_hover** - Stop scrolling when mouse hovers
- **clickable** - Emit event when clicked
- **separator** - Text between repeated content (e.g., " • ")

### Dynamic Updates

| Event | Helper | Description |
|-------|--------|-------------|
| `toolbar:marquee-set-content` | `Marquee.update_payload()` | Update entire marquee content, speed, pause |
| `toolbar:marquee-set-item` | `TickerItem.update_payload()` | Update individual items by ticker ID |

In [ ]:
# Import Marquee and TickerItem components
from pywry import Marquee, TickerItem
import random

# Create ticker items - each can be updated individually by its ticker ID
stock_items = [
    TickerItem(ticker="AAPL", text="AAPL $185.50", class_name="ticker-up"),
    TickerItem(ticker="GOOGL", text="GOOGL $142.20"),
    TickerItem(ticker="MSFT", text="MSFT $415.80", class_name="ticker-down"),
    TickerItem(ticker="AMZN", text="AMZN $178.25", class_name="ticker-up"),
]

# Build marquee content from ticker items
stock_ticker = Marquee(
    text=" • ".join(item.build_html() for item in stock_items),
    speed=20,
    pause_on_hover=True,
    component_id="stock-ticker",
)


def on_simulate_price(data, event_type, label):
    """Update a random stock price."""
    item = random.choice(stock_items)
    change = random.uniform(-5, 5)
    price = random.uniform(100, 500)
    arrow = "▲" if change > 0 else "▼"

    # update_payload() returns (event_name, payload) tuple
    event, payload = item.update_payload(
        text=f"{item.ticker} ${price:.2f} {arrow}",
        class_add="ticker-up" if change > 0 else "ticker-down",
        class_remove="ticker-down" if change > 0 else "ticker-up",
    )
    marquee_widget.emit(event, payload)


def on_pause_ticker(data, event_type, label):
    event, payload = stock_ticker.update_payload(paused=True)
    marquee_widget.emit(event, payload)


def on_resume_ticker(data, event_type, label):
    event, payload = stock_ticker.update_payload(paused=False)
    marquee_widget.emit(event, payload)


def on_speed_change(data, event_type, label):
    event, payload = stock_ticker.update_payload(speed=data.get("value", 20))
    marquee_widget.emit(event, payload)


# All toolbars - no custom HTML needed!
marquee_header = Toolbar(position="header", items=[stock_ticker])

marquee_controls = Toolbar(
    position="top",
    items=[
        Button(label="📊 Update Price", event="ticker:simulate", variant="neutral"),
        Button(label="⏸", event="ticker:pause", variant="secondary"),
        Button(label="▶️", event="ticker:resume", variant="secondary"),
        SliderInput(
            label="Speed:",
            event="ticker:speed",
            value=20,
            min=5,
            max=40,
            step=5,
            show_value=True,
        ),
    ],
    style="justify-content: center;",
)

marquee_footer = Toolbar(
    position="footer",
    items=[
        Div(
            content="<code>TickerItem.update_payload()</code> → <code>toolbar:marquee-set-item</code>",
            style="width: 100%; text-align: center; color: var(--pywry-text-secondary);",
        ),
    ],
)

marquee_widget = app.show(
    "",  # No custom HTML!
    title="Stock Ticker with TickerItem",
    toolbars=[marquee_header, marquee_controls, marquee_footer],
    callbacks={
        "ticker:simulate": on_simulate_price,
        "ticker:pause": on_pause_ticker,
        "ticker:resume": on_resume_ticker,
        "ticker:speed": on_speed_change,
    },
    height=150,
)

In [ ]:
# News ticker with static auto-cycling (no threading needed!)
news_items = [
    "📈 Markets rally on positive economic data",
    "🔔 Fed holds rates steady at 5.25%",
    "📊 Tech earnings exceed expectations",
    "🌍 Global trade talks progress well",
    "💡 AI investments surge 40% YoY",
]

# Create a static marquee with auto-cycling items
news_ticker = Marquee(
    text=news_items[0],  # Initial display
    items=news_items,  # All items to cycle through
    speed=4,  # Cycle every 4 seconds
    behavior="static",  # No scrolling, content stays in place
    component_id="news-ticker",
)

# All toolbars - no custom HTML or threading needed!
news_header = Toolbar(position="header", items=[news_ticker])

news_controls = Toolbar(
    position="top",
    items=[
        Div(
            content="<span style='color: var(--pywry-text-secondary);'>Auto-cycles every 4 seconds</span>",
            style="text-align: center; width: 100%;",
        ),
    ],
    style="justify-content: center;",
)

news_footer = Toolbar(
    position="footer",
    items=[
        Div(
            content="<code>behavior='static'</code> + <code>items=[...]</code> = auto-cycling with no event handlers",
            style="width: 100%; text-align: center; color: var(--pywry-text-secondary);",
        ),
    ],
)

news_widget = app.show(
    "",  # No custom HTML!
    title="📰 News Ticker Demo (Static Auto-Cycle)",
    toolbars=[news_header, news_controls, news_footer],
    callbacks={},
    height=150,
)

---

## 8. All Toolbar Input Components - Complete Showcase

This example shows **every toolbar component** using only Python - no custom JavaScript or HTML needed.
The entire UI is built declaratively with PyWry's toolbar system.

### Built-in System Events Demonstrated

| Event | Usage | Description |
|-------|-------|-------------|
| `pywry:update-theme` | Theme toggle (☀️) | Switches between dark/light mode |
| `pywry:inject-css` | Accent color select | Dynamically injects CSS to change accent color |
| `pywry:set-style` | Title size & Labels selects | Updates inline styles by id or CSS selector |
| `pywry:set-content` | Footer display | Updates element innerHTML from Python |

In [ ]:
# State to display current values
component_values = {}
current_theme = "dark"  # Track current theme


def make_handler(name):
    """Create a handler that updates the display with the component's value."""

    def handler(data, event_type, label):
        # Extract the relevant value(s) from the event data
        if "value" in data:
            component_values[name] = data["value"]
        elif "values" in data:
            component_values[name] = data["values"]
        elif "start" in data and "end" in data:
            component_values[name] = f"{data['start']} - {data['end']}"
        elif "btn" in data:
            component_values[name] = data["btn"]
        else:
            component_values[name] = "clicked"

        # Build display text for footer using built-in pywry:set-content
        parts = [f"<strong>{k}:</strong> {v}" for k, v in component_values.items()]
        components_widget.emit(
            "pywry:set-content", {"id": "values-display", "html": " | ".join(parts)}
        )

    return handler


def on_theme_toggle(data, event_type, label):
    """Toggle between dark and light mode using built-in pywry event."""
    global current_theme
    current_theme = "light" if current_theme == "dark" else "dark"
    components_widget.emit("pywry:update-theme", {"theme": current_theme})


def on_title_size(data, event_type, label):
    """Change the title size using built-in pywry:set-style event."""
    sizes = {"sm": "16px", "md": "20px", "lg": "26px"}
    size = data.get("value", "md")
    # Use built-in pywry:set-style event to update element styles
    components_widget.emit(
        "pywry:set-style",
        {"id": "demo-title", "styles": {"fontSize": sizes.get(size, "20px")}},
    )


def on_label_style(data, event_type, label):
    """Change all component label styles using built-in pywry:set-style event."""
    style_map = {
        "normal": {"fontWeight": "400", "fontStyle": "normal"},
        "semi": {"fontWeight": "500", "fontStyle": "normal"},
        "bold": {"fontWeight": "700", "fontStyle": "normal"},
        "italic": {"fontWeight": "400", "fontStyle": "italic"},
    }
    style = data.get("value", "normal")
    # Use built-in pywry:set-style event to update all labels
    components_widget.emit(
        "pywry:set-style",
        {
            "selector": ".pywry-input-label",
            "styles": style_map.get(style, style_map["normal"]),
        },
    )


def on_accent_color(data, event_type, label):
    """Change the accent color using built-in pywry:inject-css event."""
    colors = {
        "blue": "#0078d4",
        "green": "#28a745",
        "purple": "#6f42c1",
        "orange": "#fd7e14",
        "pink": "#e91e63",
    }
    color = data.get("value", "blue")
    accent = colors.get(color, colors["blue"])

    # Use built-in pywry:inject-css to dynamically inject CSS
    # The 'id' allows replacing the same style block on subsequent calls
    components_widget.emit(
        "pywry:inject-css",
        {
            "id": "custom-accent-color",
            "css": f"""
            :root {{
                --pywry-accent: {accent} !important;
                --pywry-accent-hover: {accent}dd !important;
            }}
            .pywry-btn-neutral {{
                background: {accent} !important;
            }}
            .pywry-btn-neutral:hover {{
                background: {accent}dd !important;
            }}
        """,
        },
    )


# Header toolbar with title and theme toggle
components_header = Toolbar(
    position="header",
    items=[
        Div(
            content="<h3 id='demo-title' style='margin: 0;'>🧩 All Toolbar Components</h3>",
            style="flex: 1;",
        ),
        Select(
            label="Accent:",
            event="demo:accent",
            options=[
                Option(label="Blue", value="blue"),
                Option(label="Green", value="green"),
                Option(label="Purple", value="purple"),
                Option(label="Orange", value="orange"),
                Option(label="Pink", value="pink"),
            ],
            selected="blue",
        ),
        Select(
            label="Title:",
            event="demo:title_size",
            options=[
                Option(label="SM", value="sm"),
                Option(label="MD", value="md"),
                Option(label="LG", value="lg"),
            ],
            selected="md",
        ),
        Select(
            label="Labels:",
            event="demo:label_style",
            options=[
                Option(label="Normal", value="normal"),
                Option(label="Semi", value="semi"),
                Option(label="Bold", value="bold"),
                Option(label="Italic", value="italic"),
            ],
            selected="normal",
        ),
        Button(
            label="☀️",
            event="demo:theme",
            variant="ghost",
            component_id="theme-toggle-btn",
        ),
    ],
)

# Top toolbar with text, number, and date inputs
inputs_row = Toolbar(
    position="top",
    items=[
        TextInput(
            label="Text:",
            event="demo:text",
            value="Hello",
            placeholder="Type here...",
        ),
        SecretInput(
            label="Secret:",
            event="demo:secret",
            placeholder="API key...",
        ),
        NumberInput(
            label="Number:",
            event="demo:number",
            value=42,
            min=0,
            max=100,
            step=1,
        ),
        DateInput(
            label="Date:",
            event="demo:date",
        ),
    ],
)

# TextArea row - multi-line resizable input
textarea_row = Toolbar(
    position="top",
    items=[
        TextArea(
            label="Notes:",
            event="demo:textarea",
            placeholder="Enter multi-line text here...\nSupports copy/paste!",
            rows=2,
            cols=120,
            resize="both",
            max_height="150px",
        ),
    ],
)

# Second row with select, multi-select
selects_row = Toolbar(
    position="top",
    items=[
        Select(
            label="Select:",
            event="demo:select",
            options=[
                Option(label="Option A", value="a"),
                Option(label="Option B", value="b"),
                Option(label="Option C", value="c"),
            ],
            selected="a",
        ),
        MultiSelect(
            label="Multi:",
            event="demo:multi",
            options=[
                Option(label="Red", value="red"),
                Option(label="Green", value="green"),
                Option(label="Blue", value="blue"),
            ],
            selected=["red"],
        ),
    ],
)

# Third row with sliders and range
sliders_row = Toolbar(
    position="top",
    items=[
        SliderInput(
            label="Slider:",
            event="demo:slider",
            value=50,
            min=0,
            max=100,
            step=5,
            show_value=True,
        ),
        RangeInput(
            label="Range:",
            event="demo:range",
            min=0,
            max=100,
            start=20,
            end=80,
            show_value=True,
        ),
    ],
)

# Fourth row with toggle, checkbox, and horizontal radio
booleans_row = Toolbar(
    position="top",
    items=[
        Toggle(label="Toggle:", event="demo:toggle", value=True),
        Div(
            content="<span class='pywry-input-label'>Check:</span>",
            style="margin-right: 4px;",
        ),
        Checkbox(label="", event="demo:checkbox", value=False),
        RadioGroup(
            label="Radio:",
            event="demo:radio",
            options=[
                Option(label="A", value="a"),
                Option(label="B", value="b"),
                Option(label="C", value="c"),
            ],
            selected="a",
            direction="horizontal",
        ),
    ],
)

# Fifth row with TabGroups
tabs_row = Toolbar(
    position="top",
    items=[
        TabGroup(
            label="View:",
            event="demo:tabs",
            options=[
                Option(label="Table", value="table"),
                Option(label="Chart", value="chart"),
                Option(label="Map", value="map"),
            ],
            selected="table",
        ),
        TabGroup(
            label="Size:",
            event="demo:tabsize",
            options=["SM", "MD", "LG"],
            selected="MD",
            size="sm",
        ),
    ],
)

# Right sidebar with vertical radio group
right_sidebar = Toolbar(
    position="right",
    style="padding: 8px 12px;",
    items=[
        Div(
            content="<span class='pywry-input-label'>Priority</span>",
            style="margin-bottom: 4px;",
        ),
        RadioGroup(
            event="demo:priority",
            options=[
                Option(label="Low", value="low"),
                Option(label="Medium", value="med"),
                Option(label="High", value="high"),
            ],
            selected="med",
            direction="vertical",
        ),
    ],
    collapsible=True,
)

# Bottom toolbar with buttons (all variants)
buttons_row = Toolbar(
    position="top",
    items=[
        Div(
            content="<span class='pywry-input-label'>Variants:</span>",
            style="margin-right: 4px;",
        ),
        Button(
            label="Primary",
            event="demo:btn",
            data={"btn": "primary"},
            variant="primary",
        ),
        Button(
            label="Secondary",
            event="demo:btn",
            data={"btn": "secondary"},
            variant="secondary",
        ),
        Button(
            label="Neutral",
            event="demo:btn",
            data={"btn": "neutral"},
            variant="neutral",
        ),
        Button(label="Ghost", event="demo:btn", data={"btn": "ghost"}, variant="ghost"),
        Button(
            label="Outline",
            event="demo:btn",
            data={"btn": "outline"},
            variant="outline",
        ),
        Button(
            label="Danger", event="demo:btn", data={"btn": "danger"}, variant="danger"
        ),
        Button(
            label="Warning",
            event="demo:btn",
            data={"btn": "warning"},
            variant="warning",
            description="Be Careful",
        ),
        Button(label="⚙", event="demo:btn", data={"btn": "icon"}, variant="icon"),
    ],
)

# Size variants row
sizes_row = Toolbar(
    position="top",
    items=[
        Div(
            content="<span class='pywry-input-label'>Sizes</span>",
            style="margin-right: 4px;",
        ),
        Button(
            label="XS",
            event="demo:btn",
            data={"btn": "xs"},
            variant="neutral",
            size="xs",
        ),
        Button(
            label="SM",
            event="demo:btn",
            data={"btn": "sm"},
            variant="neutral",
            size="sm",
        ),
        Button(
            label="Default",
            event="demo:btn",
            data={"btn": "default"},
            variant="neutral",
        ),
        Button(
            label="LG",
            event="demo:btn",
            data={"btn": "lg"},
            variant="neutral",
            size="lg",
        ),
        Button(
            label="XL",
            event="demo:btn",
            data={"btn": "xl"},
            variant="neutral",
            size="xl",
        ),
    ],
)

# Footer with live status display
components_footer = Toolbar(
    position="footer",
    items=[
        Div(
            content="<span id='values-display'><em>Interact with components above...</em></span>",
            style="color: var(--pywry-text-secondary); width: 100%; text-align: center;",
        ),
    ],
)

components_widget = app.show(
    "",  # No main content area needed, toolbars act as rows and columns.
    title="All Components Demo",
    toolbars=[
        components_header,
        inputs_row,
        textarea_row,
        selects_row,
        sliders_row,
        booleans_row,
        tabs_row,
        buttons_row,
        sizes_row,
        right_sidebar,
        components_footer,
    ],
    callbacks={
        "demo:text": make_handler("Text"),
        "demo:secret": make_handler("Secret"),
        "demo:textarea": make_handler("TextArea"),
        "demo:number": make_handler("Number"),
        "demo:date": make_handler("Date"),
        "demo:select": make_handler("Select"),
        "demo:multi": make_handler("Multi"),
        "demo:slider": make_handler("Slider"),
        "demo:range": make_handler("Range"),
        "demo:toggle": make_handler("Toggle"),
        "demo:checkbox": make_handler("Checkbox"),
        "demo:radio": make_handler("Radio"),
        "demo:tabs": make_handler("Tabs"),
        "demo:tabsize": make_handler("TabSize"),
        "demo:priority": make_handler("Priority"),
        "demo:btn": make_handler("Button"),
        "demo:theme": on_theme_toggle,
        "demo:title_size": on_title_size,
        "demo:label_style": on_label_style,
        "demo:accent": on_accent_color,
    },
    height=500,
)

---

## Summary

### Built-in PyWry Events (No Custom JS Required!)

| Event | Description | Payload |
|-------|-------------|---------|
| `pywry:set-content` | Update element HTML/text | `{id, html}` or `{id, text}` or `{selector, html}` |
| `pywry:set-style` | Update element CSS styles | `{id, styles}` or `{selector, styles}` |
| `pywry:inject-css` | Add/replace CSS dynamically | `{id, css}` |
| `pywry:update-theme` | Switch dark/light mode | `{theme: "dark"\|"light"}` |

### Toolbar-Specific Events

| Event | Description | Payload |
|-------|-------------|---------|
| `toolbar:marquee-set-content` | Update marquee text/speed | `{componentId, text?, speed?, paused?}` |
| `toolbar:marquee-set-item` | Update a TickerItem | `{ticker, text?, className?, classAdd?, classRemove?}` |

### Python Helper Methods

```python
# Marquee updates
event, payload = marquee.update_payload(text="New content", speed=15)
widget.emit(event, payload)

# TickerItem updates (individual items in marquee)
event, payload = ticker_item.update_payload(text="AAPL $200", class_add="ticker-up")
widget.emit(event, payload)
```

### Toolbar Components

| Component | Event Data | Description |
|-----------|------------|-------------|
| `Button` | `{componentId, ...data}` | Click triggers event |
| `TextInput` | `{componentId, value}` | Debounced text input |
| `TextArea` | `{componentId, value}` | Multi-line resizable text input |
| `SecretInput` | `{componentId, value}` | Password/API key input with toggle |
| `NumberInput` | `{componentId, value}` | Numeric with min/max/step |
| `Select` | `{componentId, value}` | Dropdown selection |
| `MultiSelect` | `{componentId, values}` | Checkbox group dropdown |
| `SliderInput` | `{componentId, value}` | Single value slider |
| `RangeInput` | `{componentId, start, end}` | Dual-handle range slider |
| `DateInput` | `{componentId, value}` | Date picker |
| `Toggle` | `{componentId, value}` | Boolean switch (on/off) |
| `Checkbox` | `{componentId, value}` | Boolean checkbox |
| `RadioGroup` | `{componentId, value}` | Radio buttons (single selection) |
| `TabGroup` | `{componentId, value}` | Tab-style selection (single selection) |
| `Div` | (custom) | Container for HTML/children |
| `Marquee` | (custom event) | Scrolling text ticker |
| `TickerItem` | (use in Marquee) | Updateable item in marquee |

### Boolean Components

| Component | Description |
|-----------|-------------|
| `Toggle` | Sliding switch for on/off states |
| `Checkbox` | Standard checkbox with label |
| `RadioGroup` | Radio buttons with `direction="horizontal"` or `"vertical"` |

### Selection Components

| Component | Description |
|-----------|-------------|
| `RadioGroup` | Radio buttons - use for few options in form context |
| `TabGroup` | Tab-style buttons - use for view/mode switching |

### Button Variants

| Variant | Description |
|---------|-------------|
| `primary` | Theme-aware (dark in light mode, light in dark mode) |
| `secondary` | Subtle background, theme-aware |
| `neutral` | **Always blue accent** - use for primary actions |
| `ghost` | Transparent background |
| `outline` | Bordered, transparent background |
| `danger` | Red - for destructive actions |
| `warning` | Orange - for caution/warning actions |
| `icon` | Ghost style, square aspect ratio for icon-only buttons |

### Button Sizes

| Size | Description |
|------|-------------|
| `xs` | Extra small (11px font, 2px/6px padding) |
| `sm` | Small (12px font, 4px/8px padding) |
| (default) | Normal size (13px font, 4px/8px padding) |
| `lg` | Large (15px font, 10px/20px padding) |
| `xl` | Extra large (16px font, 14px/28px padding) |

### Toolbar Options

| Option | Type | Description |
|--------|------|-------------|
| `position` | `str` | "top", "bottom", "left", "right", "inside", "header", "footer" |
| `collapsible` | `bool` | Add collapse/expand toggle |
| `resizable` | `bool` | Enable drag-to-resize |
| `class_name` | `str` | Custom CSS class |
| `style` | `str` | Inline CSS styles |

### HtmlContent for Custom Styling

```python
from pywry import HtmlContent

# Combine HTML with inline CSS for hover effects, animations, etc.
widget = app.show(
    HtmlContent(
        html=Div(...).build_html(),
        inline_css=".my-class:hover { transform: scale(1.05); }"
    ),
    ...
)
```

### 2-Way Communication Pattern

```python
# Python → JavaScript (use built-in events!)
widget.emit("pywry:set-content", {"id": "my-element", "html": "New content"})
widget.emit("pywry:set-style", {"id": "my-element", "styles": {"color": "red"}})

# JavaScript → Python (automatic via toolbar events)
callbacks={"event:name": handler_function}
```

### When You Might Need Custom JS

Custom JavaScript is rarely needed, but may be useful for:
- **Complex animations** (beyond CSS transitions)
- **Third-party library integration** (charts, maps, etc.)
- **Custom DOM manipulation** not covered by built-in events

For most use cases including dynamic lists, grids, and content updates, use the built-in `pywry:set-content` event!